In [ ]:
#Send campaign
import requests
import json
import base64
import hashlib
import pandas as pd


class Customer_io():
    def __init__(self):
        self.api_auth = '1c03f7a99821ad19037c6726689fce29'
        self.track_api = '998e14fb966d15171526'
        self.track_cred ='4007a0eacc29eaf377f0'
        self.track_auth='{0}:{1}'.format(self.track_api,self.track_cred)
        
    def get_users_segment(self, segment:int) -> list:
        url='https://api.customer.io/v1/segments/{0}/membership'.format(segment)

        headers = { 'Authorization': "Bearer {0}".format(self.api_auth) }

        response = requests.request("GET", url, headers=headers).json()
        return response['ids']
    
    def del_users_segment(self,segment:int,users:list) -> bool:
        url = "https://track.customer.io/api/v1/segments/{0}/remove_customers?id_type=id".format(segment)
        headers = {
          'Authorization': 'Basic {0}'.format(base64.b64encode(self.track_auth.encode()).decode('utf-8')),
          'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=json.dumps({"ids": old_users}))
        if response.ok:
            return True
        else:
            print(response.text)
            return False
    
    def add_users_segment(self,segment:int,df:pd.DataFrame,attributes:list=[]) -> bool:
        
        df.drop_duplicates(subset=['email'], keep='first')
        
        df['md5em']=df.email.apply(lambda x: (hashlib.md5(x.encode())).hexdigest())
        headers = {
                  'Authorization': 'Basic {0}'.format(base64.b64encode(self.track_auth.encode()).decode('utf-8')),
                  'content-type': 'application/json'
                }
        if len(attributes)>0:
            #Add attributes
            payload = dict.fromkeys(attributes)
            for i in range(len(df)):
                url = "https://track.customer.io/api/v1/customers/{0}".format(df.md5em.iloc[i])
                print(df.email.iloc[i])
                for j in payload:
                    payload[j]= df[j].iloc[i]
                print(payload)    
                response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
                if not response.ok:
                    print('Something went wrong when adding attributes, please check: {0}'.format(response.text))
                    return False
                
        url = "https://track.customer.io/api/v1/segments/{0}/add_customers".format(segment)
        response = requests.request("POST", url, headers=headers, data=json.dumps({"ids": df.md5em.to_list()}))
        if response.ok:
            print('Users uploaded')
            return True
        else:
            print('Something went wrong when adding users, please check:{0}'.format(response.text))
            return False
    
#Connect to Klar DWH
def read_yaml_file(yaml_file):
    """ load yaml cofigurations """
    config = None
    try: 
        with open (yaml_file, 'r') as f:
            config = yaml.safe_load(f)
            
    except:
        raise FileNotFoundError('Couldnt load the file')

    return config

def get_db_conn(creds_file):
    """ Get an authenticated psycopg db connection, given a credentials file"""
    creds = read_yaml_file(creds_file)['db']

    connection = psycopg2.connect(
        user=creds['user'],
        password=creds['pass'],
        host=creds['host'],
        port=creds['port'],
        database=creds['db']
    )

    return connection
    

In [ ]:
#Put SQL queries here 
data="select * from public.user_kyc_"
data=pd.read_csv('Undeliverable08-11-2022.csv')
data.head()


In [ ]:
import time

cio= Customer_io()

if len(data)>0:
    #segment id here
    segment = 762
    #Attributes if exist here.
    attribute=['a','b']
    old_users = cio.get_users_segment(segment)
    print(old_users)
    #Logics 
    if len(old_users)>0:
        cio.del_users_segment(segment,old_users)
    cio.add_users_segment(segment,data,attribute)
    
    time.sleep(4)
    new_users = cio.get_users_segment(segment)
    print(len(new_users))
    if len(new_users) != len(data):
        print('Not all users are in the campaign, please verify')
else:
    print("No campaign")